## References

* [Adult Data Set](https://archive.ics.uci.edu/ml/datasets/Adult) (UCI)

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

from common import Dataset, column_variants, encode_categorical, load_data
from sklearn.preprocessing import StandardScaler

np.random.seed(0)

In [ ]:
# https://github.com/statsmodels/statsmodels/issues/3931
from scipy import stats
stats.chisqprob = lambda a, b: stats.chi2.sf(a, b)

In [ ]:
data = load_data('data/train.csv')
data.info()

In [ ]:
data.dropna(inplace=True)

redundant = [
    'CapitalGain',
    'CapitalLoss',
    'EducationNumber',
    'FinalSamplingWeight',
]

categorical = [
    'Education',
    'MaritalStatus',
    'NativeCountry',
    'Occupation',
    'Race',
    'Relationship',
    'Sex',
    'WorkClass',
]

data.drop(redundant, axis=1, inplace=True)
data.reset_index(drop=True, inplace=True)

for name in categorical:
    data = encode_categorical(data, name)

data.info()

In [ ]:
dataset = Dataset(data)
x = sm.add_constant(dataset.x_train)
y = dataset.y_train

In [ ]:
model = sm.OLS(y, x)

result = model.fit()
result.summary()